In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import os

# Import ORDO data
ordo = pd.read_csv(os.path.join(os.getcwd(), 'rawdl_20240310', 'ordo.csv'))

In [ ]:
# Filter the gene
ordo_gene = ordo[ordo['http://www.w3.org/2004/02/skos/core#notation'].isnull()]
ordo_gene = ordo[ordo['http://www.orpha.net/ORDO/symbol'].notnull()]

# Filter the disease
ordo_disease = ordo[ordo['http://www.w3.org/2004/02/skos/core#notation'].notnull()]

# Filter the definition
ordo_other = ordo[~ordo['Class ID'].isin(ordo_gene['Class ID']) & ~ordo['Class ID'].isin(ordo_disease['Class ID'])]

print('Gene : ', len(ordo_gene))
print('Disease : ', len(ordo_disease))
print('Other : ', len(ordo_other))
print('Total : ', len(ordo))

In [13]:
# # View and Iterate the data in ORDO
# columns = ordo.columns

# for item in ordo.head().iterrows() :
#     print('====================')
#     for column in columns :
#         print(column, ' : ', item[1][column])

Class ID  :  http://www.orpha.net/ORDO/Orphanet_478664
Preferred Label  :  Hereditary sensory and autonomic neuropathy type 8
Synonyms  :  HSAN8|Hereditary sensory and autonomic neuropathy type VIII
Definitions  :  A rare autosomal recessive hereditary sensory and autonomic neuropathy characterized by congenital impaired sensation of acute or inflammatory pain in combination with an inability to identify noxious heat or cold, leading to numerous painless mutilating lesions and injuries. Further manifestations are absence of corneal reflexes resulting in corneal scarring, reduced sweating and tearing, and recurrent skin infections. Large-fiber sensory modalities such as light touch, vibration, and proprioception are normal.
Obsolete  :  False
CUI  :  nan
Semantic Types  :  nan
Parents  :  http://www.orpha.net/ORDO/Orphanet_377788|http://www.orpha.net/ORDO/Orphanet_557493
biomarker tested in  :  nan
candidate gene tested in  :  nan
curator_inference  :  nan
disease-causing germline mutat

In [ ]:
clean_ordo = pd.DataFrame()
clean_ordo['ORPHA'] = ordo['http://www.w3.org/2004/02/skos/core#notation']
clean_ordo['label'] = ordo['Preferred Label']
clean_ordo